In [ ]:
import numpy as np

In [ ]:
import keras
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt",shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [ ]:
tokenizer.texts_to_sequences(["First"])

In [ ]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

In [ ]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [ ]:
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text]))-1

In [ ]:
import tensorflow as tf
train_size = dataset_size*90//100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [ ]:
n_steps = 100
window_length = n_steps+1
dataset = dataset.window(window_length,shift=1,drop_remainder=True)

In [ ]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [ ]:
batch_size=32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows:(windows[:,:-1],windows[:,1:]))

In [ ]:
dataset = dataset.map(
    lambda X_batch,Y_batch: (tf.one_hot(X_batch,depth=max_id),Y_batch)
)
dataset = dataset.prefetch(1)

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128,return_sequences=True,input_shape=[None,max_id],dropout=0.2,recurrent_dropout=0.2),
    keras.layers.GRU(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation="softmax"))
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam")
history = model.fit(dataset,epochs=20)

In [ ]:
def preprocess(texts):
  X = np.array(tokenizer.texts_to_sequences(texts))-1
  return tf.one_hot(X,max_id)

In [ ]:
X_new = preprocess(["how are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred+1)[0][-1]

In [ ]:
def next_char(text,temperature=1):
  X_new = preprocess([text])
  y_proba = model.predict(X_new)[0,-1:,:]
  rescaled_logits = tf.math.log(y_proba)/temperature
  char_id = tf.random.categorical(rescaled_logits,num_samples=1)+1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
def complete_text(text,nchars=50,temperature=1):
  for _ in range(n_chars):
    text += next_char(text,temperature)
  return text

In [ ]:
print(complete_text("t",temperature=0.2))

In [ ]:
print(complete_text("w",temperature=1))

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length,shift=n_steps,drop_remainder=True)
dataset = dataset.flat_map(lambda window:window.batch(window_length))
dataset = dataset.batch(1)

dataset = dataset.map(lambda X_Y: (tf.one_hot(X_Y[0], depth=max_id), X_Y[1]))

dataset = dataset.prefetch(1)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Input(batch_shape=(batch_size, None, max_id)),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(
        keras.layers.Dense(max_id, activation="softmax"))
])


In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.model.reset_states()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam")

In [ ]:
model.fit(dataset,epochs=50,callbacks=[ResetStatesCallback])

In [ ]:
(X_train,y_train),(X_test,y_test) = keras.datasets.imdb.load_data()
X_train[0][:10]

In [ ]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_+3:word for word,id_ in word_index.items()}
for id_,token in enumerate(("<pad>","<sos>","<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

In [ ]:
import tensorflow_datasets as tfds
datasets,info = tfds.load("imdb_reviews",as_supervised=True,with_info=True)
train_size = info.splits["train"].num_examples

In [ ]:
def preprocess(X_batch,y_batch):
    X_batch = tf.strings.substr(X_batch,0,300)
    X_batch = tf.strings.regex_replace(X_batch,b"<br\\s*/?>",b" ")
    X_batch = tf.strings.regex_replace(X_batch,b"[^a-zA-Z']",b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"),y_batch

In [ ]:
from collections import Counter
vocabulary = Counter()
for X_batch,y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [ ]:
vocabulary.most_common()[:3]

In [ ]:
vocab_size = 1000
truncated_vocabulary = [
    word for word,count in vocabulary.most_common()[:vocab_size]
]

In [ ]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary),dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words,word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init,num_oov_buckets)

In [ ]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

In [ ]:
def encode_words(X_batch,y_batch):
    return table.lookup(X_batch),y_batch
train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [ ]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size+num_oov_buckets,embed_size,input_shape=[None]),
    keras.layers.GRU(128,return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1,activation="sigmoid")
])
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
history = model.fit(train_set,epochs=5)

In [ ]:
K = keras.backend
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(
    lambda inputs: K.not_equal(inputs, 0),
    output_shape=inputs.shape
)(inputs)
z = keras.layers.Embedding(vocab_size+num_oov_buckets,embed_size)(inputs)
z = keras.layers.GRU(128,return_sequences=True)(z,mask=mask)
z = keras.layers.GRU(128)(z,mask=mask)
outputs = keras.layers.Dense(1,activation="sigmoid")(z)
model = keras.Model(inputs=[inputs],outputs=[outputs])

In [ ]:
import tensorflow_hub as hub
model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                         dtype=tf.string,input_shape=[],output_shape=[50]),
    keras.layers.Dense(128,activation="relu"),
    keras.layers.Dense(1,activation="sigmoid")
])
model.compile(loss="binary_crossentropy",optimizer="adam",
              metrics=["accuracy"])


In [ ]:
datasets,info = tfds.load("imdb_reviews",as_supervised=True,with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].batch(batch_size).prefetch(1)
history = model.fit(train_set,epochs =5)

In [1]:
import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=None,dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None],dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[],dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size,embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512,return_state=True)
encoder_outputs,state_h,state_c = encoder(encoder_embeddings)
encoder_state = [state_h,state_c]

sampler = tfa.seq2seq.sampler.trainingSampler()

decoder_cell = keras.layers.LSTM(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell,sampler,output_layer=output_layer)
final_outputs,final_state,final_sequence_lengths = decoder(
    decoder_embeddings,initial_state = encoder_state,sequence_length=sequence_lengths
)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)
model = keras.Model(inputs=[encoder_inputs,decoder_inputs,sequence_lengths],
                    outputs=[Y_proba])

ModuleNotFoundError: No module named 'tensorflow_addons'

In [2]:
import tensorflow as tf
print(tf.__version__)


2.19.0
